**Projektni zadatak**

Kreirana je neuronska mreza nad Covertype skupom podataka.
Za rad sa neuronskim mrezama je koristena bibliotrka Pytorch.
Ukoliko se koristi Google Collab ova biblioteka je automatski
instalirana. Za instalaciju na Linux operativnom sistemu potrebno je u komandnoj liniji pokrenuti sljedecu naredbu:


```
~$ pip3 install torch torchvision torchaudio
```



In [1]:
import torch
import torch.nn as nn

In [2]:
from sklearn.datasets import fetch_covtype #importovanje zadanog skupa podataka

covtype = fetch_covtype()
x = covtype.data #ulazne vrijednosti
y = covtype.target #ciljane vijerdnosti

Analizirajuci Covertype skup podataka zakljucujemo da imamo ulazne podatke prikazane kao vektor duzine 54 pa iz toga zakljucujemo da su nam potrebna 54 ulazna neurona. S obzirom da je nad podacima potrebno izvrsiti klasifikaciju i posto imamo 7 klasa, onda imamo i 7 izlaznih cvorova u neuronskoj mrezi

In [3]:
class CovertypeClasificator(nn.Module):
  #Konstruktor
  def __init__(self):
    super().__init__() #pozivanje konstruktora roditeljeske klase

    #input sloj

    self.input_layer = nn.Linear(54, 128) #definisanje ulaznog sloja cvorova
    self.act_input = nn.ReLU() #koristim ReLU aktivaciou funkciju zbog njene jednostavnosti i izbjegavanja problema gradijentnog nestajanja
    #self.drop_input = nn.Dropout(p=0.25) #preko dropout-a smanjujemo overfitting tako sto se tokom treninga sa odredjenom vjerovatnocom iskljucuju cvorovi

    #sloj 1

    self.layer1 = nn.Linear(128, 64)
    self.act1 = nn.ReLU()
    self.drop1 = nn.Dropout(p=0.22) #preko dropout-a smanjujemo overfitting tako sto se tokom treninga sa odredjenom vjerovatnocom iskljucuju cvorovi

    #sloj 2

    self.layer2 = nn.Linear(64, 64)
    self.act2 = nn.ReLU()
    self.drop2 = nn.Dropout(p=0.22)

    #output sloj

    self.output = nn.Linear(64, 7)

  #metoda za propagaciju unaprijed (forward pass) (x(ulazni podaci) se propagira kroz slojeve mreze)
  def forward(self, x):
    x = self.act_input(self.input_layer(x))
    x = self.act1(self.layer1(x))
    x = self.drop1(x)
    x = self.act2(self.layer2(x))
    x = self.drop2(x)
    x = self.output(x)
    return x

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, shuffle=True, random_state=23)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=23)
#dijelimo podatke na trening, testne i validacione. Negdje je praksa da testni podaci cine od 20 do 30% ukupnog broja zadataka
#oznacili smo shuffle kao true da bi se podaci nasumicno rasporedili prije podjele
#kao seed za random brojeve smo izabrali konkretan broj da bismo uvijek imali istu sekvencu nasumicnih brojeva

In [5]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

#Bitno je da znamo da su nam podaci iz sklearn.datasets dati kao numpy nizovi (np.ndarray) a Pytorch radi sa
#tenzorima pa je potrebno konvertovati ulazne podatke.

x_train_t = torch.tensor(x_train).float()
x_test_t = torch.tensor(x_test).float()
x_val_t = torch.tensor(x_val).float()
y_train_t = torch.tensor(y_train - 1).long() #Neophodno je da stavimo y_train - 1 jer klase idu od 0 do 6 a ne od 1 do 7
y_test_t = torch.tensor(y_test - 1).long() #long je pogodno koristiti kod klasifikacije
y_val_t = torch.tensor(y_val - 1).long()

#TensorDataset se koristi da se ulazni podaci i ciljne klase grupisu zajedno u jedan dataset
train_dataset = TensorDataset(x_train_t, y_train_t) #grupisemo trening podatke
test_dataset = TensorDataset(x_test_t, y_test_t) #grupisemo test podatke
val_dataset = TensorDataset(x_val_t, y_val_t) #grupisemo validacione podatke

#Klasa Dataloader se koristi za iteriranje kroz trening i test datasetove.
#Podaci se grupisu u batcheve(skupovi uzoraka iz dataset-a koji se koristi za treniranje
#ili evaluaciju modela) određene velicine.
#batch_size je postavljen na 40 tako da ce svaka iteracija imati 40 uzoraka
#shuffle u train loader-u je stavljen kao True tako da ce se podaci mijesati prije
#svake epohe. Za test i validacioni skup nije neophodno sshuffle staviti true jer se testiranje vrsi nad nevidjenim podacima
train_loader = DataLoader(train_dataset, batch_size=40, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=40)
val_loader = DataLoader(val_dataset, batch_size=40)

In [6]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu") # koristimo gpu samo ako je dostupan, inace koristimo cpu
print(device)
net = CovertypeClasificator().to(device) #kreieamo mrezu i premjestimo je na izabrani uredjaj

cuda:0


In [7]:
loss_fn = nn.CrossEntropyLoss() #funkcija greske koja daje dobre rezultate za klasifikaciju
EPOCHS = 35 #epoha je jedan prelazak kroz kompletan skup trening podataka

#optimizator se koristi za azuriranje podataka mreze tokom treninga
#prvi argument je skup parametara mreže koje će optimizator azurirati
#drugi argument predstavlja stopu ucenja dok treci predstavlja faktor
#regulacije koji sprjecava preveliko prilagodjavanje trening podacima
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)

#test_model(net, loss_fn, test_loader)

In [8]:
# TRAIN FUNKCIJA
def train_model(net, loss_fn, optimizer, EPOCHS):
  net.train()  # stavljamo mrežu u mod rada za trening
  for i in range(EPOCHS): #Vrsimo treniranje onoliko puta koliko je epoha definisano
    print(f"Izvrsavanje epohe {i + 1}") #Kontrolni ispis da znamo da se izvrsava posto trening traje dugo (Nema nikakvog uticaja na logiku)
    for x, y in train_loader: #prolazimo kroz trening podatke
      x = x.to(device) #Prebacujemo podatke na dostupan uredjaj
      y = y.to(device)
      preds = net(x) #dobijanje predikcije koriscenjem mreze i ulaznih podataka x
      loss = loss_fn(preds, y) #Izracunavanje gubitka izmedju predikcija i ciljnih vrijednosti koriscenjem funkcije
      optimizer.zero_grad() #postavljanje gradijenata svih parametara mreze na nulu
      loss.backward() #izračunavanje gradijenta gubitka loss unazad kroz mrežu odnosno narodski receno propagacija unazad
      optimizer.step() #optimizacijski korak - azuriranje parametara na osnovu dobijenih gradijenata

In [9]:
def train_model_with_early_stopping(net, loss_fn, optimizer, train_loader, val_loader, EPOCHS): #implementacija druge tacke - rano stopiranje tokom treninga
  best_val_loss = float('inf') #varijabla za pracenje najbolje greske, u pocetku joj je dodjeljeno +beskonacno
  increasing_epochs = 0 #Brojac uzastopnih epoha u kojima greska raste

  for epoch in range(EPOCHS):
    net.train()
    train_loss = 0 #vrijednost gubitka tokom treninga
    for x, y in train_loader: #sve identicno kao kod obicnog treninga
      x = x.to(device)
      y = y.to(device)
      preds = net(x)
      loss = loss_fn(preds, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader) #izracunavanje prosjecne greske po uzorku u trening skupu

    net.eval() #stavljanje mreze u evalucioni mod
    total_val_loss = 0 #ukupan gubitak nad validacionim skupom
    with torch.no_grad(): #ne racunamo gradijent jer nam on nije potreban tokom testa i validacije
      for x_val, y_val in val_loader: #sljedeci blok koda je identican onom za trening s tim da se ne racuna gradijent i ne propagira greska unazad (ne vrsi se trening mreze)
        x_val = x_val.to(device)
        y_val = y_val.to(device)
        preds_val = net(x_val)
        val_loss = loss_fn(preds_val, y_val)
        total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader) #izracunavanje prosjecne greske u validacionom skupu
    print(f'Epoha {epoch+1}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}') #Kontrolni ispis

    if best_val_loss > val_loss: #provjeravamo da li se greska na validacionom skupu povecava
      best_val_loss = val_loss #imamo novu najmanju gresku
      increasing_epochs = 0 #ukoliko je brojac bio uvecavan, restartujemo ga
    else:
      increasing_epochs += 1 #ukoliko greska raste, inkrementujemo brojac

    if increasing_epochs > 2: #ukoliko je prekoracen broj dozvoljenih epoha sa greskom
      print("Early stopping - zavrsava se trening")
      break #obustavljamo dalje izvrsavanje epoha


In [10]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
import warnings

def test_model(net, loss_fn, test_loader): #definisemo funkciju za testiranje modela
    total_loss = 0 #ovu varijablu cemo koristiti za sumiranje ukupnih gubitaka
    net.eval()
    y_true = [] #inicijalizacija liste za stvarne vrijednosti
    y_pred = [] #inicijalizacija liste za predvidjene vrijednosti, koristicemo ih za racunanje metrika

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            preds = net(x)
            _, predicted = torch.max(preds.data, 1) #Koristimo _ da bismo ignorisali povratnu vrijednost - indeks maksimalne vrijednosti
                                                    #jer nam on nije potreban. Koristimo funkciju max jer nam treba najveca vjerovatnoca pjavljivanja klase
            loss = loss_fn(preds, y) #racunanje gubitka uz pomoc predikcija i stvarnih vrijednosti
            total_loss += loss.item() #akumuliranje pojedinacnih gubitaka iz test skupa
            y_true.extend(y.cpu().numpy()) #prosirujemo liste vrijednostima ali ih prvo pretvaramo iz tenzora u niz jer izabrane funkcije za metrike rade sa nizovima
            y_pred.extend(predicted.cpu().numpy())

    print(total_loss / len(test_loader)) #prosjecan gubitak nad testnim skupom podataka
    accuracy = accuracy_score(y_true, y_pred) #racunanje tacnosti
    precision = precision_score(y_true, y_pred, average='macro', zero_division = 0.0) #racunanje preciznosti
    recall = recall_score(y_true, y_pred, average='macro') #racunanje odziva

    print(f'Tacnost: {accuracy:.2f}')
    print(f'Preciznost: {precision:.2f}')
    print(f'Ukupan odziv: {recall:.2f}')

    # Racunanje odziva za svaku klasu posebno
    class_recall = recall_score(y_true, y_pred, average=None)
    for i, recall in enumerate(class_recall):
        print(f'Odziv za klasu {i}: {recall:.2f}')




In [11]:
#train_model(net, loss_fn, optimizer, EPOCHS)
test_model(net, loss_fn, test_loader)

21.728117665530302
Tacnost: 0.01
Preciznost: 0.03
Ukupan odziv: 0.13
Odziv za klasu 0: 0.00
Odziv za klasu 1: 0.00
Odziv za klasu 2: 0.12
Odziv za klasu 3: 0.78
Odziv za klasu 4: 0.00
Odziv za klasu 5: 0.00
Odziv za klasu 6: 0.00


In [16]:
train_model_with_early_stopping(net, loss_fn, optimizer, train_loader, val_loader, EPOCHS)

Epoha 1: Train Loss: 0.6620, Val Loss: 0.7134
Epoha 2: Train Loss: 0.6569, Val Loss: 0.6227
Epoha 3: Train Loss: 0.6522, Val Loss: 0.6195
Epoha 4: Train Loss: 0.6512, Val Loss: 0.6182
Epoha 5: Train Loss: 0.6500, Val Loss: 0.6467
Epoha 6: Train Loss: 0.6502, Val Loss: 0.6198
Epoha 7: Train Loss: 0.6459, Val Loss: 0.6050
Epoha 8: Train Loss: 0.6459, Val Loss: 0.6181
Epoha 9: Train Loss: 0.6426, Val Loss: 0.6101
Epoha 10: Train Loss: 0.6423, Val Loss: 0.6271
Early stopping - zavrsava se trening


In [17]:
test_model(net, loss_fn, test_loader)

0.6271645752672057
Tacnost: 0.73
Preciznost: 0.75
Ukupan odziv: 0.51
Odziv za klasu 0: 0.63
Odziv za klasu 1: 0.90
Odziv za klasu 2: 0.72
Odziv za klasu 3: 0.57
Odziv za klasu 4: 0.13
Odziv za klasu 5: 0.45
Odziv za klasu 6: 0.18
